In [1]:
import gurobipy as gp
from whygreedy import Compound
from pymatgen.analysis.phase_diagram import Composition, PDEntry, GrandPotentialPhaseDiagram, GrandPotPDEntry
from whygreedy.schema import gen_random_data

def find_lp_open(reactant: Compound, products: list[Compound], open_elements_data:dict[str, float]):
    if len(products) == 0:
        return [], - reactant.formation_energy_per_atom

    elements_in_products = []
    for product in products:
        elements_in_products += product.elements
    elements_in_products = set(elements_in_products)
    elements_in_constraints = sorted(set(reactant.elements).intersection(elements_in_products))
    open_elements = elements_in_products.difference(set(reactant.elements))

    # init gurobi model, suppress output
    with gp.Env(empty=True) as env:
        env.setParam('OutputFlag', 0)
        env.setParam('LogToConsole', 0)
        env.start()
        with gp.Model(env=env) as m:

            # add variables
            x = []
            for iproduct, product in enumerate(products):
                x_i = m.addVar(name=str(iproduct))
                x.append(x_i)

            # add stoi constraints
            c = []
            for e in elements_in_constraints:
                element_sum = 0
                for i, x_i in enumerate(x):
                    try:
                        composition = products[i].normalized_formula[e]
                    except KeyError:
                        continue
                    element_sum += composition * x_i
                c_e = m.addConstr(element_sum == reactant.normalized_formula[e], name=e)
                c.append(c_e)
            # add non-negative constraints
            for i, x_i in enumerate(x):
                c_e = m.addConstr(x_i >= 0, name="x_{}".format(i))
                c.append(c_e)

            # objective function
            objective = 0
            for x_i, product in zip(x, products):
                openterm = 0
                for openele in open_elements:
                    openterm += product.normalized_formula[openele] * open_elements_data[openele]
                objective += x_i * (product.formation_energy_per_atom - openterm)

            m.setObjective(objective, gp.GRB.MINIMIZE)
            m.optimize()
            return [v.x for v in m.getVars()], m.objVal - reactant.formation_energy_per_atom

def gpd_decomp(reactant:Compound, oxides:list[Compound], open_elements_data):
    entries = [PDEntry(Composition(e.normalized_formula), e.formation_energy_per_atom) for e in [reactant,] + oxides]
    gpd = GrandPotentialPhaseDiagram(entries, open_elements_data)
    sol = gpd.get_decomposition(Composition(reactant.normalized_formula))
    delta = - reactant.formation_energy_per_atom
    for pd, x_i in sol.items():
        pd: GrandPotPDEntry
        delta += pd.energy_per_atom * x_i
        delta -= open_elements_data["O"] * x_i * pd.composition["O"]
        # real x_i for compositions has a factor of 1/pd.composition.num_atoms
    return delta

In [2]:
import random
from tqdm import tqdm
for seed in tqdm(range(1000)):
    random.seed(seed)
    open_data = {"O": random.randint(-100, 0)}
    reactant, oxides = gen_random_data(["C", "Si", "Ge", "Sn"], 5, seed)
    delta_pmg = gpd_decomp(reactant, oxides, open_data)
    _, delta_gb = find_lp_open(reactant, oxides, open_data)
    assert abs(delta_gb - delta_pmg) < 1e-6

100%|██████████| 1000/1000 [00:26<00:00, 37.98it/s]


In [3]:
%timeit delta = gpd_decomp(reactant, oxides, open_data)

15.9 ms ± 230 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
%timeit _, delta = find_lp_open(reactant, oxides, open_data)

1.68 ms ± 64.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
